In [47]:
import pandas as pd
from cade.cade import CADE
from gensim.models.word2vec import Word2Vec
import os
import glob
import json
import numpy as np

In [13]:
df_list = pd.DataFrame()
for filename in glob.glob('./data_final/*.json'):
    print(filename)
    with open(filename, 'r') as f:
        json_load = json.loads(f.read())
        df_list = df_list.append(pd.DataFrame.from_records(json_load, index="_id"), ignore_index = True)

./data_final/scraping_data_Slate.json
./data_final/scraping_data_NYTimes.json
./data_final/scraping_data_Breitbart.json
./data_final/scraping_data_CNN.json
./data_final/scraping_data_ABC.json
./data_final/scraping_data_Federalist.json
./data_final/scraping_data_Wikipedia.json
./data_final/scraping_data_NewsMax.json


In [15]:
df_list = df_list[['Title', "Date", "Link", "Paragraphs", "Authors"]]

In [69]:
def create_newssite(link = None):
        if link is np.nan:
            return("Wikipedia")
        elif "slate" in link:
            return("Slate")
        elif "https://www.nytimes.com" in link:
            return("New York Times")
        elif "https://www.breitbart.com" in link:
            return("Breitbart")
        elif "https://www.cnn.com" in link:
            return("CNN")
        elif "abcnews" in link:
            return("ABC News")
        elif "https://thefederalist.com" in link:
            return("The Federalist")
        elif "https://www.newsmax.com" in link:
            return("News Max")
        else:
            return("Unknown")

In [73]:
df_list["Newssite"] = df_list["Link"].apply(create_newssite)

if len(df_list[df_list["Newssite"][:] == "Unknown"]) == 0:
    print("AVVOCATO CI SIAMO")

AVVOCATO CI SIAMO


/Users/leonardoalchieri/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [75]:
with open("./data_safe.csv", 'w') as file:
    df_list.to_csv(file)